In [ ]:
!pip install biopython scikit-learn --quiet
import pandas as pd
import numpy as np
from Bio import SeqIO
from collections import Counter
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import RadiusNeighborsClassifier
from sklearn.metrics import classification_report, accuracy_score
from sklearn.feature_selection import SelectKBest, mutual_info_classif
from random import shuffle
def load_sequences(file_paths, window=150, stride=50):
    sequences, labels = [], []
    for label, path in file_paths.items():
        for record in SeqIO.parse(path, "fasta"):
            seq = str(record.seq).replace("*", "")
            for i in range(0, len(seq) - window + 1, stride):
                frag = seq[i:i+window]
                if len(frag) == window:
                    sequences.append((frag, label))
    shuffle(sequences)
    fragments, labels = zip(*sequences)
    return list(fragments), list(labels)
def get_kmers(seq, k=3):
    return [seq[i:i+k] for i in range(len(seq)-k+1)]
def build_kmer_features(sequences, k=3):
    all_kmers = set()
    kmer_counts = []
    for seq in sequences:
        kmers = get_kmers(seq, k)
        count = Counter(kmers)
        kmer_counts.append(count)
        all_kmers.update(count)
    all_kmers = sorted(list(all_kmers))
    kmer_index = {kmer: idx for idx, kmer in enumerate(all_kmers)}
    features = np.zeros((len(sequences), len(all_kmers)), dtype=np.float32)
    for i, count in enumerate(kmer_counts):
        for kmer, freq in count.items():
            if kmer in kmer_index:
                features[i, kmer_index[kmer]] = freq
    return features, all_kmers
file_paths = {
    'Zika': '/content/drive/MyDrive/sagar/Zika_200.fasta',
    'Ebola': '/content/drive/MyDrive/sagar/Ebola_200.fasta',
    'SARS_CoV_2': '/content/drive/MyDrive/sagar/SARS_CoV_2_200.fasta',
    'Influenza_A': '/content/drive/MyDrive/sagar/Influenza_A_200.fasta',
    'Influenza_B': '/content/drive/MyDrive/sagar/Influenza_B_200.fasta',
    'Tuberculosis': '/content/drive/MyDrive/sagar/Tuberculosis_200.fasta',
    'Human': '/content/drive/MyDrive/sagar/Human_200.fasta'
}
data, labels = load_sequences(file_paths, window=150, stride=50)
print(f" Augmented sequences: {len(data)}")
X, kmer_names = build_kmer_features(data, k=3)
y = pd.factorize(pd.Series(labels))[0]
selector = SelectKBest(mutual_info_classif, k=200)
X_selected = selector.fit_transform(X, y)
X_train, X_test, y_train, y_test = train_test_split(
    X_selected, y, test_size=0.2, stratify=y, random_state=42
)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
radius_model = RadiusNeighborsClassifier(
    radius=5.0,
    outlier_label='most_frequent',
    weights='distance'
)
radius_model.fit(X_train_scaled, y_train)
y_pred_radius = radius_model.predict(X_test_scaled)
acc_radius = accuracy_score(y_test, y_pred_radius)
print(f"\n Radius Neighbors Accuracy: {acc_radius * 100:.2f}%")
print("\n Radius Neighbors Classification Report:\n")
print(classification_report(y_test, y_pred_radius))

 Augmented sequences: 9820

 Radius Neighbors Accuracy: 93.79%

 Radius Neighbors Classification Report:

              precision    recall  f1-score   support

           0       0.92      0.88      0.90        67
           1       1.00      0.91      0.95       426
           2       0.88      1.00      0.94       829
           3       0.97      0.93      0.95       237
           4       1.00      0.95      0.98       151
           5       1.00      0.80      0.89       252
           6       1.00      1.00      1.00         2

    accuracy                           0.94      1964
   macro avg       0.97      0.92      0.94      1964
weighted avg       0.94      0.94      0.94      1964

